In [134]:
import requests
from bs4 import BeautifulSoup
import sqlite3
import re
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [135]:
from bs4 import BeautifulSoup
import re
from uuid import uuid4

In [136]:
class Crawler:

    def get_html(self):
        pass

    def get_card_info(self):
        pass

    def get_links(self, page):
        pass

In [137]:
class CrawlerEda(Crawler):

    def get_html(self, page):
        chrome_options = Options()
        chrome_options.add_argument("--disable-extensions")
        chrome_options.add_argument("--disable-popup-blocking")

        browser = webdriver.Chrome(options=chrome_options)
        url = f"https://eda.ru/recepty?page={page}"
        #browser.implicitly_wait(30)
        browser.get(url)
        wait = WebDriverWait(browser, 10)
        soup = BeautifulSoup(browser.page_source, 'html.parser')
        return soup

    def get_card_info(self, number):
        chrome_options = Options()
        chrome_options.add_argument("--disable-extensions")
        chrome_options.add_argument("--disable-popup-blocking")

        browser = webdriver.Chrome(options=chrome_options)
        action = ActionChains(browser)
        count = 0

        #linker = linkers[number]['link']
        #browser.get(linker)
        wait = WebDriverWait(browser, 10)
        soup_card = BeautifulSoup(browser.page_source, 'html.parser')
        return soup_card

    def get_links(self, page):
        links = []
        for i in range(1, page):
            html = CrawlerEda().get_html(i)
            links.extend(ParserEda(html).get_cards())
            print(links)
        return links

In [138]:
class Parser:

    def __init__(self, html):
        self.html = html

    def get_cards(self):
        pass

In [139]:
class ParserEda(Parser):

    def get_cards(self):
        linkers = []
        for recipe in self.html.findAll('div', class_='emotion-1f6ych6'):
            try:
                link = "https://eda.ru"+recipe.find('a', class_='emotion-12sjte8').get('href')
                photo = recipe.find('img', class_='emotion-1bi124x').get('src')
                #print(link)
                #print(photo)
                linkers.append(Link(re.findall(r'\b\d+\b', link)[0], link, photo))
            except AttributeError:
                print("There's no item with that code")
        print(len(linkers))
        return linkers


In [140]:
class Recipe:
    def __init__(self, key, name_dish, count_portion, cook_time, ingredients, manual):
        self.id = key
        self.name_dish = name_dish
        self.count_portion = count_portion
        self.cook_time = cook_time
        self.ingredients = ingredients
        self.manual = manual

In [141]:
class Ingredient:
    def __init__(self, key, name_ingr):
        self.id = key
        self.name_ingr = name_ingr

In [142]:
class Calories:
    def __init__(self, key, calories, proteins, fats, carbohydrates):
        self.id = key
        self.calories = calories
        self.proteins = proteins
        self.fats = fats
        self.carbohydrates = carbohydrates

In [143]:
class Link:
    def __init__(self, key, link, photo):
        self.id = key
        self.link = link
        self.photo = photo

In [175]:
class DBHelper:

    def __init__(self, db_name = "recipe.db"):
        self.dbname = f"sqlite3:/{db_name}"
        self.conn = sqlite3.connect(db_name)

    def simple_create_table(self, table_name, primary_key, params):
        column = ""
        for i in range(len(params) - 1):
            column = str(params[i]) + " TEXT, "
        column += str(params[len(params) - 1]) + " TEXT "
        new_table = f"CREATE TABLE IF NOT EXISTS {table_name} ({primary_key} TEXT PRIMARY KEY , {column})"
        self.conn.execute(new_table)
        self.conn.commit()

    def create_table(self, table_name, primary_key, foreign_table_name, foreign_col, another_foreign_col, params):
        for i in len(params) - 1:
            column = column + params[i] + " TEXT, "
        column += params[len(params)] + " TEXT "

        foreign_key = f"FOREIGN KEY ({foreign_col})  REFERENCES {foreign_table_name} ({another_foreign_col})"

        new_table = f"CREATE TABLE IF NOT EXISTS {table_name} ({primary_key} TEXT PRIMARY KEY , {column}, {foreign_key})"
        self.conn.execute(new_table)
        self.conn.commit()

    def insert(self, table_name, args):
        for i in len(args) - 1:
            arguments = args[i] + ", "
        arguments += args[len(args) - 1]

        new_insert = f"INSERT INTO {table_name} VALUES ({arguments})"
        self.conn.execute(new_insert)
        self.conn.commit()

    def print_info(self, table_name):
        new_get = f"SELECT * FROM {table_name}"
        cursor = self.conn.execute(new_get)
        return [row[0] for row in cursor]

    #def update(self, flat_id, updates):


    #def delete(self, flat_id):


In [145]:
linkers = CrawlerEda().get_links(5)
print(len(linkers))
for i in linkers:
    print(i.id, i.link, i.photo)

14
[<__main__.Link object at 0x0000021705C966D0>, <__main__.Link object at 0x0000021705CC4460>, <__main__.Link object at 0x0000021706895B50>, <__main__.Link object at 0x0000021706895670>, <__main__.Link object at 0x0000021706895790>, <__main__.Link object at 0x00000217068958B0>, <__main__.Link object at 0x00000217068956D0>, <__main__.Link object at 0x0000021706895C40>, <__main__.Link object at 0x00000217058E9820>, <__main__.Link object at 0x00000217058E9760>, <__main__.Link object at 0x00000217058E9550>, <__main__.Link object at 0x00000217058E9C40>, <__main__.Link object at 0x00000217058E92B0>, <__main__.Link object at 0x00000217058E9100>]
14
[<__main__.Link object at 0x0000021705C966D0>, <__main__.Link object at 0x0000021705CC4460>, <__main__.Link object at 0x0000021706895B50>, <__main__.Link object at 0x0000021706895670>, <__main__.Link object at 0x0000021706895790>, <__main__.Link object at 0x00000217068958B0>, <__main__.Link object at 0x00000217068956D0>, <__main__.Link object at 0

In [177]:
DBHelper("recipe.db").simple_create_table("link", "id", ["link", "photo"])

In [166]:
DBHelper("recipe.db").print_info("link")

[]

In [ ]:
linkers = CrawlerEda().get_links(5)
print(len(linkers))
for i in linkers:
    DBHelper().insert("link", [i.id, i.link, i.photo])